En este primer bloque de código se importa el archivo A1.4VinoTinto.csv utilizando la función read_csv de la librería pandas. Posteriormente, se imprimen las dimensiones del dataframe con el atributo shape y, mediante la función head, se muestran las primeras 5 filas del archivo.


In [1]:
import pandas as pd
df = pd.read_csv("A1.4VinoTinto.csv")
print("Dimensiones del data frame: ",df.shape)
print(df.head(5))

Dimensiones del data frame:  (1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2      

En el siguiente bloque de código se divide el conjunto de datos en entrenamiento y prueba utilizando la función train_test_split de la librería sklearn.model_selection, asignando un 80% de los datos al entrenamiento y 20% a los de prueba. Posteriormente, se separan las variables independientes (X_train y X_test) de la variable dependiente (Y_train y Y_test), donde la columna calidad se considera como la variable de salida. Finalmente, se imprimen las dimensiones de los subconjuntos de entrenamiento y prueba, así como el total para revisar que todos los datos se tomen en cuenta.

In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size = 0.8)
X_train = train.drop('calidad', axis=1)
Y_train = train['calidad']
X_test = test.drop('calidad', axis=1)
Y_test = test['calidad']
print("Train:", train.shape)
print("Test:",test.shape)
print("Total:",len(train)+len(test))

Train: (1279, 12)
Test: (320, 12)
Total: 1599


En el siguiente bloque de código se utiliza SequentialFeatureSelector de la librería mlxtend.feature_selection para generar la metodología de selección hacia adelante. Se establece que el número de características seleccionadas varíe entre 2 y 8, utilizando como métrica de evaluación el coeficiente de determinación (r2) y validación cruzada con 10 particiones. Finalmente, se muestran en pantalla los índices y los nombres de las características seleccionadas mediante este procedimiento.

In [3]:
import mlxtend.feature_selection as mlx
import sklearn.linear_model as sk
sfs_forward = mlx.SequentialFeatureSelector(estimator=sk.LinearRegression(), k_features=(2,8), forward=True, scoring='r2', cv=10)
sfs_forward.fit(X_train, Y_train)
print("Características seleccionadas forward (índices):", sfs_forward.k_feature_idx_)
print("Características seleccionadas forward (nombres):", sfs_forward.k_feature_names_)

Características seleccionadas forward (índices): (1, 2, 4, 5, 6, 8, 9, 10)
Características seleccionadas forward (nombres): ('acidezVolatil', 'acidoCitrico', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


En el siguiente bloque de código se construye un modelo de regresión lineal utilizando únicamente las características seleccionadas en el método de selección hacia adelante. Primero, se generan los conjuntos de entrenamiento y prueba (X_train_forward y X_test_forward) a partir de las variables seleccionadas. Posteriormente, se entrena el modelo con los datos de entrenamiento y se realizan predicciones sobre los datos de prueba. Finalmente, se evalúa el desempeño del modelo mediante el coeficiente de determinación R^2, mostrando el valor obtenido en la consola.


In [4]:
import sklearn.metrics as met
X_train_forward = X_train[list(sfs_forward.k_feature_names_)]
X_test_forward = X_test[list(sfs_forward.k_feature_names_)]
model_forward = sk.LinearRegression()
model_forward.fit(X_train_forward, Y_train)
Y_pred_forward = model_forward.predict(X_test_forward)
r2_forward = met.r2_score(Y_test, Y_pred_forward)
print("R^2 (predicción forward): ",r2_forward)

R^2 (predicción forward):  0.32944855869752465


En el siguiente bloque de código se vuelve a aplicar el método SequentialFeatureSelector, pero ahora utilizando el método de selección hacia atrás.  
En este caso, se emplea como estimador un modelo de regresión lineal, seleccionando entre 2 y 5 características. El parámetro forward=False indica que el proceso se realiza de forma regresiva. Para el fit se utilizan ya las variables de la selección hacia adelante.  La evaluación del modelo se realiza con la métrica r2 y validación cruzada con 10 particiones. Finalmente, se imprimen en pantalla los índices y nombres de las características seleccionadas.


In [5]:
sfs_backward = mlx.SequentialFeatureSelector(estimator=sk.LinearRegression(), k_features=(2,5), forward=False, scoring='r2', cv=10)
sfs_backward.fit(X_train_forward, Y_train)
print("Características seleccionadas backward (índices):", sfs_backward.k_feature_idx_)
print("Características seleccionadas backward (nombres):", sfs_backward.k_feature_names_)

Características seleccionadas backward (índices): (0, 2, 4, 6, 7)
Características seleccionadas backward (nombres): ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')


En este último bloque de código se construye un modelo de regresión lineal utilizando únicamente las características seleccionadas mediante el método de selección hacia atrás.  Primero, se generan los conjuntos de entrenamiento y prueba (X_train_backward y X_test_backward) a partir de las variables elegidas en dicho proceso. Posteriormente, el modelo se entrena con los datos de entrenamiento y se realizan predicciones sobre los datos de prueba.  Finalmente, se calcula el coeficiente de determinación R^2 para evaluar el desempeño del modelo, mostrando en pantalla el valor obtenido.

In [6]:
X_train_backward = X_train[list(sfs_backward.k_feature_names_)]
X_test_backward = X_test[list(sfs_backward.k_feature_names_)]
model_backward = sk.LinearRegression()
model_backward.fit(X_train_backward, Y_train)
Y_pred_backward = model_backward.predict(X_test_backward)
r2_backward = met.r2_score(Y_test, Y_pred_backward)
print("R^2 (predicción backward): ",r2_backward)

R^2 (predicción backward):  0.3228529612466712


Al comparar los valores de R^2 obtenidos con los métodos de selección hacia adelante y hacia atrás, se observa que el método hacia adelante tiene mejores resultados al analizar las distintas corridas del código ( R^2 más cercana a 1). Sin embargo, es importante mencionar que el método hacia atrás se basa en los resultados del método hacia adelante, ya que busca reducir la cantidad de variables utilizadas. Esto se puede observar en la diferencia en el número de características seleccionadas por cada método.

Es decir, si bien el modelo de selección hacia adelante es mejor, el método hacia atrás obtiene resultados similares pero con una cantidad menor de características, por lo que dependiendo del uso del modelo puede ser más conveniente utilizar solo el método hacia adelante o también considerar el método hacia atrás. Al final del día, el método hacia atrás tiende a mostrar un desempeño menor porque utiliza menos características, lo cual en algunos casos puede ser beneficioso para el manejo de datos.